In [ ]:
import yaml
import os
from collections import defaultdict
os.chdir('/home/mrsergazinov/TabLLM/')
from train_eval import set_seed, load_data, preprocess_data, train_and_evaluate_model

In [ ]:
from encoders.numEncoders import (
    FourierFeatures,
    FourierFeaturesSq, 
    BinningFeatures, 
    ComboFeatures,
    SquareScalingFeatures,
)
SEEDS = [42, 7, 123, 2020, 999, 77, 88, 1010, 2021, 3030]
for seed in SEEDS:
        set_seed(seed)

        params = {
                'dataset_name': 'otto_group',
                'model_name': 'MLP',
                'num_encoder': 'FourierFeatures',
                'num_encoder_trainable': False,
                'scaler': 'SquareScalingFeatures',
                'n_run': 1,
                'random_state': seed,
                'test_size': 0.2,
        }

        if params['num_encoder'] is not None:
                path = f'configs/{params["model_name"]}_{params["num_encoder"]}_{params["scaler"]}_{params["num_encoder_trainable"]}_{params["dataset_name"]}.yaml'
        else:
                path = f'configs/{params["model_name"]}_{params["dataset_name"]}.yaml'
        with open(path, 'r') as file:
                config = yaml.safe_load(file)

        X, y, task_type = load_data(params)
        (y_train, 
        y_val, y_test, 
        X_train_num, 
        X_val_num, 
        X_test_num, 
        X_train_cat, 
        X_val_cat, 
        X_test_cat) = preprocess_data(X, y, task_type, params)
        metric = train_and_evaluate_model(
                X_train_num=X_train_num,
                X_test_num=X_test_num,
                X_train_cat=X_train_cat,
                X_test_cat=X_test_cat,
                y_train=y_train,
                y_test=y_test,
                task_type=task_type,
                config=config,
                params=params,
                verbose_training=False,
        )
        print(metric)